In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Process
import fcntl

In [2]:


data_dir = 'data'

transition_period = 300 #frames
out_of_view_accept_period = 30 #frames
bb_threshold = True
bb_threshold_x = 15
bb_threshold_y = 60

carla_raw_output_dir = '/home/tung/carlasim/data_archived/output_tracks'




if bb_threshold:
    extracted_dir_path = '/home/tung/carlasim/data_archived/extracted_tracks_{}_{}_bb_{}_{}'.format(transition_period, out_of_view_accept_period, bb_threshold_x, bb_threshold_y)
else:
    extracted_dir_path = '/home/tung/carlasim/data_archived/extracted_tracks_{}_{}'.format(transition_period, out_of_view_accept_period)

extracted_tracks_for_training_dir = os.path.join(extracted_dir_path, 'individuals-for-train')
extracted_tracks_for_simulation_dir = os.path.join(extracted_dir_path,'e2e-for-simulation')

for duh in [extracted_dir_path, extracted_tracks_for_simulation_dir, extracted_tracks_for_training_dir]:
    if not os.path.exists(duh):
        os.mkdir(duh)

num_of_camera = 18
print(extracted_tracks_for_simulation_dir, extracted_tracks_for_training_dir)

/home/tung/carlasim/data_archived/extracted_tracks_300_30_bb_15_60/e2e-for-simulation /home/tung/carlasim/data_archived/extracted_tracks_300_30_bb_15_60/individuals-for-train


for d in [extracted_tracks_for_training_dir, extracted_tracks_for_simulation_dir]:
    for f in os.listdir(d):
        os.remove(os.path.join(d, f))

In [9]:
def format_e2e_traj(e2e_traj):
    """Laying an end-to-end track flat in the frame-by-frame format

    A typical end-to-end track would look like this:
        ...
        [[97958,  1225,   129],     Number of frame in carla, x coordinate, y coordinate
         [97959,  1225,   129],
         [97960,  1226,   128],
         [97961,  1226,   128],
         [97962,  1226,   127]]) 10 15 98]  Source camera, destination (next) camera, number of transition frames
        ...
        [[...,  ...,   ...],     
         [...,  ...,   ...]]) a b c]  
        ...
        [[...,  ...,   ...],     
         [...,  ...,   ...]]) -1 -1 -1]        
    
    And it seems difficult to be process efficiently. Thus, we try to lay it flat frame-by-frame:
        ...
        [97958,  1225,   129],     Number of frame in carla, x coordinate, y coordinate
        [97959,  1225,   129],
        [97960,  1226,   128],
        [97961,  1226,   128],
        [97962,  1226,   127]]) 10 15 98]  Source camera, destination (next) camera, number of transition frames
        ...
        [...,  ...,   ...]     
        [...,  ...,   ...]  
        ...
        [...,  ...,   ...],     
        [...,  ...,   ...]

    Args:
        e2e_track: An e2e track in the form mentioned above

    Return:
               
    """
    frame_idx = -1
    new_traj = []
    for idx, traj in enumerate(e2e_traj):
        
        curr_cam = traj[1]
        next_cam = traj[2]
        trans_time = traj[3]
        traj = traj[0]
        cam_first_seem_frame = traj[0][0]
        if frame_idx != -1:
            while frame_idx <= (cam_first_seem_frame - total_first_seen_frame):
                frame_idx += 1
                new_traj.append((frame_idx, -1, -1, -1))
        else:
            total_first_seen_frame = traj[0][0]
        
        for entry in traj:
            frame_idx += 1
            new_traj.append((frame_idx, entry[1], entry[2], curr_cam))

    return np.array(new_traj, dtype = object)
            


In [10]:



def extraction(start, end):
    def get_cam_entry_list(df, index, num_of_cam):
        cam_entry_list = {}
        for cam_num in range(num_of_cam):
            cam_id = 'Camera ' + str(cam_num)
            current_entry = df.iloc[index][cam_id]
            current_entry = current_entry.replace('(', '')
            current_entry = current_entry.replace(')', '')
            
            if (current_entry != '-1, -1'):
                # if cam_num == 9:
                #     print(current_entry)
                temp = current_entry.split(', ')
                if bb_threshold:
                    if ((float(temp[3]) > bb_threshold_x) or (float(temp[4]) > bb_threshold_y)):
                        frame, coor_x, coor_y = (int(x) for x in temp[0:3])
                        cam_entry_list[cam_num] = (frame, coor_x, coor_y)
                    else:
                        cam_entry_list[cam_num] = (-1, -1)   
                else: 
                    frame, coor_x, coor_y = (int(x) for x in temp[0:3])
                    cam_entry_list[cam_num] = (frame, coor_x, coor_y)
                
            else:
                cam_entry_list[cam_num] = (-1, -1)
        return cam_entry_list

    print('Process {}, start: {}, end: {}'.format(os.getpid(), start, end))

    trajectory_list = {}
    for cam_num in range(num_of_camera):
        trajectory_list[cam_num] = []

    for file_num, file_name in enumerate(tqdm(os.listdir(carla_raw_output_dir), desc = '{}-{}'.format(start, end))):
        if '_' in file_name:
            num = int(file_name.split('_')[0])
        else:
            num = int(file_name.split('.')[0])
        if ((num > end) or (num < start)):
            continue
        #for file_name in ['52_28.csv']:#['892_21.csv']: #['1150_26.csv']:
        #print(file_name)
        file_path = os.path.join(carla_raw_output_dir, file_name)
        #file_path = '/home/tung/carlasim/data/output_tracks/1303_29.csv'
        df = pd.read_csv(file_path, sep = ',', header = 0)
        currently_tracked = {}
        in_transition_period = {}
        in_out_of_view_accept_period = {}
        last_frame_num = {}
        prev_frame_num = {}
        first_frame_num = {}
        trajectory = {}
        parallel_list = {}
        e2e_trajectory_list = []
        e2e_trajectory = []
        for cam_num in range(num_of_camera):
            currently_tracked[cam_num] = False
            in_transition_period[cam_num] = False
            in_out_of_view_accept_period[cam_num] = False
            first_frame_num[cam_num] = -999999999999
            last_frame_num[cam_num] = 99999999999999
            prev_frame_num = 99999999999999
            trajectory[cam_num] = []
            parallel_list[cam_num] = {}
            for other_cam_num in range(num_of_camera):
                if other_cam_num != cam_num:
                    parallel_list[cam_num][other_cam_num] = 0
                else:
                    parallel_list[cam_num][other_cam_num] = -1
        total_num_of_frames = len(df)
        for index in range(total_num_of_frames):
            frame_num = df.iloc[index]['#']
            cam_entry_list = get_cam_entry_list(df, index, num_of_camera)

            for cam_num in range(num_of_camera):
                current_entry = cam_entry_list[cam_num]
                if current_entry == (-1, -1):
                    if (frame_num - last_frame_num[cam_num]) == 1:
                        
                        in_out_of_view_accept_period[cam_num] = True
                        in_transition_period[cam_num] = True

                            
                    if (frame_num - last_frame_num[cam_num]) > out_of_view_accept_period:
                        in_out_of_view_accept_period[cam_num] = False
                        currently_tracked[cam_num] = False
                        

                    if (frame_num - last_frame_num[cam_num]) > transition_period:
                        in_transition_period[cam_num] = False
                        in_out_of_view_accept_period[cam_num] = False

                        first_frame_num[cam_num] = -99999999999
                        last_frame_num[cam_num] = 99999999999
                        prev_frame_num = 999999999999

                        found = False
                        for e2e_traj in e2e_trajectory_list:
                            #print(e2e_traj[len(e2e_traj) - 1][2])
                            if e2e_traj[len(e2e_traj) - 1][2] == cam_num:
                                e2e_traj.append(list([np.array(trajectory[cam_num]), cam_num, -1, -1]))
                                
                        trajectory[cam_num] = []

                    
                else:
                    if not currently_tracked[cam_num]:
                        first_frame_num[cam_num] = frame_num
                        currently_tracked[cam_num] = True
                        in_out_of_view_accept_period[cam_num] = False
                        in_transition_period[cam_num] = False
                        

                    trajectory[cam_num].append((current_entry[0], current_entry[1], current_entry[2]))
                    last_frame_num[cam_num] = frame_num


                # if (cam_num == 5) and (frame_num in [179832]):
                #     print(frame_num, last_frame_num[cam_num], current_entry, currently_tracked[cam_num], in_transition_period[cam_num], in_out_of_view_accept_period[cam_num])
                # if (cam_num == 6) and (frame_num in [179832]):
                #     print(frame_num, last_frame_num[cam_num], current_entry, currently_tracked[cam_num], in_transition_period[cam_num], in_out_of_view_accept_period[cam_num])

                for other_cam_num in range(num_of_camera):
                    if cam_num != other_cam_num:
                        if (currently_tracked[cam_num] == 0) and (currently_tracked[other_cam_num] == 0):
                            parallel_list[cam_num][other_cam_num] = 0
                        if (cam_entry_list[cam_num] != (-1, -1) and cam_entry_list[other_cam_num] != (-1, -1)):
                            parallel_list[cam_num][other_cam_num] = 1
                        # if (cam_num == 6) and (other_cam_num == 5) and (frame_num in [179832]):
                        #     print(frame_num, last_frame_num[other_cam_num], current_entry, currently_tracked[other_cam_num], in_transition_period[other_cam_num], in_out_of_view_accept_period[other_cam_num])
                        if in_transition_period[cam_num]:             
                            entry = cam_entry_list[other_cam_num]
                            # if (cam_num == 6) and (other_cam_num == 5) and (frame_num in [179832]):
                            #     print(frame_num, last_frame_num[other_cam_num], current_entry, currently_tracked[other_cam_num], in_transition_period[other_cam_num], in_out_of_view_accept_period[other_cam_num])
                            #     print(parallel_list[cam_num][other_cam_num])
                            # if (other_cam_num == 6) and (frame_num in [179832]):
                            #    print(entry, currently_tracked[other_cam_num], in_transition_period[other_cam_num], in_out_of_view_accept_period[other_cam_num])
                            if (entry != (-1, -1)) and (currently_tracked[other_cam_num]) and (parallel_list[cam_num][other_cam_num] == 0):
                                # print(frame_num, last_frame_num[cam_num], cam_num, other_cam_num, entry, currently_tracked[other_cam_num])
                                #print(cam_num, other_cam_num, entry)
                                traj = list([np.array(trajectory[cam_num], dtype = int), cam_num, other_cam_num, frame_num - last_frame_num[cam_num]])
                                found = False
                                for e2e_traj in e2e_trajectory_list:
                                    if e2e_traj[len(e2e_traj) - 1][2] == cam_num:
                                        e2e_traj.append(traj)
                                        found = True
                                if not found:
                                    temp = []
                                    temp.append(traj)
                                    e2e_trajectory_list.append(temp)
                                trajectory_list[cam_num].append(np.array(traj, dtype = object))

                                currently_tracked[cam_num] = False
                                in_transition_period[cam_num] = False
                                in_out_of_view_accept_period[cam_num] = False
                                first_frame_num[cam_num] = -999999999999
                                last_frame_num[cam_num] = 99999999999999
                                prev_frame_num = 99999999999999
                                trajectory[cam_num] = []
                                break

                
                prev_frame_num = frame_num

                if index == total_num_of_frames - 1:
                    found = False
                    for e2e_traj in e2e_trajectory_list:
                        if e2e_traj[len(e2e_traj) - 1][2] == cam_num:
                            e2e_traj.append(list([np.array(trajectory[cam_num]), cam_num, -1, -1]))
        
        should_stop = False
        for index, e2e_traj in enumerate(e2e_trajectory_list):
            e2e_traj = np.array(e2e_traj, dtype = object)
            
            temp_traj = format_e2e_traj(e2e_traj)
            file_path = os.path.join(extracted_tracks_for_simulation_dir, '{}-{}.npy'.format(file_name.split('.')[0], index))

            np.save(file_path, temp_traj)

        
    # for cam_num in range(num_of_camera):
    #     if len(trajectory_list[cam_num]) > 0:
    #         temp = np.array(trajectory_list[cam_num])
    #         cam_extracted_file_path = os.path.join(extracted_tracks_for_training_dir, str(cam_num)) + '.npy'
    #         if os.path.exists(cam_extracted_file_path):
    #             with open(cam_extracted_file_path, 'rb') as f:
    #                 fcntl.flock(f, fcntl.LOCK_EX)
    #                 old_data = np.load(cam_extracted_file_path, allow_pickle = True)
    #                 temp = np.concatenate((old_data, temp), axis = 0)
    #                 fcntl.flock(f, fcntl.LOCK_UN)
    #                 f.close()
                    
            
    #         with open(cam_extracted_file_path, 'wb') as f:
    #             fcntl.flock(f, fcntl.LOCK_EX)
    #             np.save(cam_extracted_file_path, temp)
    #             fcntl.flock(f, fcntl.LOCK_UN)
    #             f.close()
      
                

In [11]:
for i in range(20):
    u = 125
    start = u * i + 1
    end = u * (i + 1)
    p = Process(target = extraction, args = (start, end))
    p.start()

Process 1470568, start: 1, end: 125
Process 1470569, start: 126, end: 250
Process 1470574, start: 251, end: 375
Process 1470579, start: 376, end: 500
Process 1470587, start: 501, end: 625
Process 1470599, start: 626, end: 750Process 1470610, start: 751, end: 875

Process 1470625, start: 876, end: 1000
Process 1470644, start: 1001, end: 1125
Process 1470659, start: 1126, end: 1250
Process 1470686, start: 1251, end: 1375
Process 1470709, start: 1376, end: 1500
Process 1470726, start: 1501, end: 1625
Process 1470754, start: 1626, end: 1750

251-375:   0%|          | 0/7597 [00:00<?, ?it/s]

1-125:   0%|          | 0/7597 [00:00<?, ?it/s]

Process 1470796, start: 1751, end: 1875


126-250:   0%|          | 0/7597 [00:00<?, ?it/s]

...

1876-2000:   0%|          | 0/7597 [00:00<?, ?it/s]

2001-2125:   0%|          | 0/7597 [00:00<?, ?it/s]

2126-2250:   0%|          | 0/7597 [00:00<?, ?it/s]

2251-2375:   0%|          | 0/7597 [00:00<?, ?it/s]

2376-2500:   0%|          | 0/7597 [00:00<?, ?it/s]

In [ ]:



def extraction(start, end):
    def get_cam_entry_list(df, index, num_of_cam):
        cam_entry_list = {}
        for cam_num in range(num_of_cam):
            cam_id = 'Camera ' + str(cam_num)
            current_entry = df.iloc[index][cam_id]
            current_entry = current_entry.replace('(', '')
            current_entry = current_entry.replace(')', '')
            
            if (current_entry != '-1, -1'):
                # if cam_num == 9:
                #     print(current_entry)
                temp = current_entry.split(', ')
                if bb_threshold:
                    if ((float(temp[3]) > bb_threshold_x) or (float(temp[4]) > bb_threshold_y)):
                        frame, coor_x, coor_y = (int(x) for x in temp[0:3])
                        cam_entry_list[cam_num] = (frame, coor_x, coor_y)
                    else:
                        cam_entry_list[cam_num] = (-1, -1)   
                else: 
                    frame, coor_x, coor_y = (int(x) for x in temp[0:3])
                    cam_entry_list[cam_num] = (frame, coor_x, coor_y)
                
            else:
                cam_entry_list[cam_num] = (-1, -1)
        return cam_entry_list

    print('Process {}, start: {}, end: {}'.format(os.getpid(), start, end))

    trajectory_list = {}
    for cam_num in range(num_of_camera):
        trajectory_list[cam_num] = []

    for file_num, file_name in enumerate(tqdm(os.listdir(raw_output_dir), desc = '{}-{}'.format(start, end))):
        if '_' in file_name:
            num = int(file_name.split('_')[0])
        else:
            num = int(file_name.split('.')[0])
        if ((num > end) or (num < start)):
            continue
        #for file_name in ['52_28.csv']:#['892_21.csv']: #['1150_26.csv']:
        #print(file_name)
        file_path = os.path.join(raw_output_dir, file_name)
        #file_path = '/home/tung/carlasim/data/output_tracks/1303_29.csv'
        df = pd.read_csv(file_path, sep = ',', header = 0)
        currently_tracked = {}
        in_transition_period = {}
        in_out_of_view_accept_period = {}
        last_frame_num = {}
        prev_frame_num = {}
        first_frame_num = {}
        trajectory = {}
        parallel_list = {}
        e2e_trajectory_list = []
        e2e_trajectory = []
        for cam_num in range(num_of_camera):
            currently_tracked[cam_num] = False
            in_transition_period[cam_num] = False
            in_out_of_view_accept_period[cam_num] = False
            first_frame_num[cam_num] = -999999999999
            last_frame_num[cam_num] = 99999999999999
            prev_frame_num = 99999999999999
            trajectory[cam_num] = []
            
            parallel_list[cam_num] = {}
            for other_cam_num in range(num_of_camera):
                if other_cam_num != cam_num:
                    parallel_list[cam_num][other_cam_num] = 0
                else:
                    parallel_list[cam_num][other_cam_num] = -1
        total_num_of_frames = len(df)
        for index in range(total_num_of_frames):
            frame_num = df.iloc[index]['#']
            cam_entry_list = get_cam_entry_list(df, index, num_of_camera)

            for cam_num in range(num_of_camera):
                current_entry = cam_entry_list[cam_num]
                if current_entry == (-1, -1):
                    if (frame_num - last_frame_num[cam_num]) == 1:
                        
                        in_out_of_view_accept_period[cam_num] = True
                        in_transition_period[cam_num] = True

                            
                    if (frame_num - last_frame_num[cam_num]) > out_of_view_accept_period:
                        in_out_of_view_accept_period[cam_num] = False
                        currently_tracked[cam_num] = False
                        

                    if (frame_num - last_frame_num[cam_num]) > transition_period:
                        in_transition_period[cam_num] = False
                        in_out_of_view_accept_period[cam_num] = False

                        first_frame_num[cam_num] = -99999999999
                        last_frame_num[cam_num] = 99999999999
                        prev_frame_num = 999999999999

                        found = False
                        for e2e_traj in e2e_trajectory_list:
                            #print(e2e_traj[len(e2e_traj) - 1][2])
                            if e2e_traj[len(e2e_traj) - 1][2] == cam_num:
                                e2e_traj.append(list([np.array(trajectory[cam_num]), -1, -1, -1]))
                                
                        trajectory[cam_num] = []
                    
                else:
                    if not currently_tracked[cam_num]:
                        first_frame_num[cam_num] = frame_num
                        currently_tracked[cam_num] = True
                        in_out_of_view_accept_period[cam_num] = False
                        in_transition_period[cam_num] = False
                        

                    trajectory[cam_num].append((current_entry[0], current_entry[1], current_entry[2]))
                    last_frame_num[cam_num] = frame_num


                # if (cam_num == 5) and (frame_num in [179832]):
                #     print(frame_num, last_frame_num[cam_num], current_entry, currently_tracked[cam_num], in_transition_period[cam_num], in_out_of_view_accept_period[cam_num])
                # if (cam_num == 6) and (frame_num in [179832]):
                #     print(frame_num, last_frame_num[cam_num], current_entry, currently_tracked[cam_num], in_transition_period[cam_num], in_out_of_view_accept_period[cam_num])

                for other_cam_num in range(num_of_camera):
                    if cam_num != other_cam_num:
                        if (currently_tracked[cam_num] == 0) and (currently_tracked[other_cam_num] == 0):
                            parallel_list[cam_num][other_cam_num] = 0
                        if (cam_entry_list[cam_num] != (-1, -1) and cam_entry_list[other_cam_num] != (-1, -1)):
                            parallel_list[cam_num][other_cam_num] = 1
                        # if (cam_num == 6) and (other_cam_num == 5) and (frame_num in [179832]):
                        #     print(frame_num, last_frame_num[other_cam_num], current_entry, currently_tracked[other_cam_num], in_transition_period[other_cam_num], in_out_of_view_accept_period[other_cam_num])
                        if in_transition_period[cam_num]:             
                            entry = cam_entry_list[other_cam_num]
                            # if (cam_num == 6) and (other_cam_num == 5) and (frame_num in [179832]):
                            #     print(frame_num, last_frame_num[other_cam_num], current_entry, currently_tracked[other_cam_num], in_transition_period[other_cam_num], in_out_of_view_accept_period[other_cam_num])
                            #     print(parallel_list[cam_num][other_cam_num])
                            # if (other_cam_num == 6) and (frame_num in [179832]):
                            #    print(entry, currently_tracked[other_cam_num], in_transition_period[other_cam_num], in_out_of_view_accept_period[other_cam_num])
                            if (entry != (-1, -1)) and (currently_tracked[other_cam_num]) and (parallel_list[cam_num][other_cam_num] == 0):
                                # print(frame_num, last_frame_num[cam_num], cam_num, other_cam_num, entry, currently_tracked[other_cam_num])
                                #print(cam_num, other_cam_num, entry)
                                traj = list([np.array(trajectory[cam_num], dtype = int), cam_num, other_cam_num, frame_num - last_frame_num[cam_num]])
                                found = False
                                for e2e_traj in e2e_trajectory_list:
                                    if e2e_traj[len(e2e_traj) - 1][2] == cam_num:
                                        e2e_traj.append(traj)
                                        found = True
                                if not found:
                                    temp = []
                                    temp.append(traj)
                                    e2e_trajectory_list.append(temp)
                                trajectory_list[cam_num].append(np.array(traj, dtype = object))

                                currently_tracked[cam_num] = False
                                in_transition_period[cam_num] = False
                                in_out_of_view_accept_period[cam_num] = False
                                first_frame_num[cam_num] = -999999999999
                                last_frame_num[cam_num] = 99999999999999
                                prev_frame_num = 99999999999999
                                trajectory[cam_num] = []
                                break

                
                prev_frame_num = frame_num

                if index == total_num_of_frames - 1:
                    found = False
                    for e2e_traj in e2e_trajectory_list:
                        if e2e_traj[len(e2e_traj) - 1][2] == cam_num:
                            e2e_traj.append(list([np.array(trajectory[cam_num]), cam_num, -1, -1]))
        for index, e2e_traj in enumerate(e2e_trajectory_list):
            e2e_traj = np.array(e2e_traj, dtype = object)
            file_path = os.path.join(extracted_tracks_for_simulation_dir, '{}-{}.npy'.format(file_name.split('.')[0], index))
            with open(file_path, 'w') as ff:
                np.save(file_path, e2e_traj)
    for cam_num in range(num_of_camera):
        if len(trajectory_list[cam_num]) > 0:
            temp = np.array(trajectory_list[cam_num])
            cam_extracted_file_path = os.path.join(extracted_tracks_for_training_dir, str(cam_num)) + '.npy'
            if os.path.exists(cam_extracted_file_path):
                with open(cam_extracted_file_path, 'rb') as f:
                    fcntl.flock(f, fcntl.LOCK_EX)
                    old_data = np.load(cam_extracted_file_path, allow_pickle = True)
                    temp = np.concatenate((old_data, temp), axis = 0)
                    fcntl.flock(f, fcntl.LOCK_UN)
                    f.close()
                    
            
            with open(cam_extracted_file_path, 'wb') as f:
                fcntl.flock(f, fcntl.LOCK_EX)
                np.save(cam_extracted_file_path, temp)
                fcntl.flock(f, fcntl.LOCK_UN)
                f.close()
                
                

In [2]:
import numpy as np

for file_name in sorted(os.listdir('/home/tung/carlasim/data_archived/extracted_tracks_300_30_bb_15_60/e2e-for-simulation')):
    a = os.path.join('/home/tung/carlasim/data_archived/extracted_tracks_300_30_bb_15_60/e2e-for-simulation', file_name)
    b = np.load(a, allow_pickle = True)
    print(b, '\n\n\n')

[[0 4 179 4]
 [1 17 178 4]
 [2 29 178 4]
 ...
 [540 2005 55 1]
 [541 2005 55 1]
 [542 2005 54 1]] 



[[0 10 178 4]
 [1 23 178 4]
 [2 35 177 4]
 ...
 [543 2005 55 1]
 [544 2005 55 1]
 [545 2005 54 1]] 



[[0 2 179 4]
 [1 15 178 4]
 [2 27 178 4]
 ...
 [543 2005 55 1]
 [544 2005 55 1]
 [545 2005 54 1]] 



[[0 7 179 4]
 [1 19 178 4]
 [2 31 178 4]
 ...
 [541 2005 55 1]
 [542 2005 55 1]
 [543 2005 54 1]] 



[[0 7 179 4]
 [1 19 178 4]
 [2 32 178 4]
 ...
 [479 2126 51 1]
 [480 2125 51 1]
 [481 2125 50 1]] 



[[0 1864 75 12]
 [1 1865 75 12]
 [2 1866 75 12]
 ...
 [663 98 624 11]
 [664 63 631 11]
 [665 28 638 11]] 



[[0 1863 75 12]
 [1 1864 75 12]
 [2 1865 75 12]
 ...
 [665 73 629 11]
 [666 38 636 11]
 [667 2 643 11]] 



[[0 1863 75 12]
 [1 1864 75 12]
 [2 1866 75 12]
 ...
 [662 99 624 11]
 [663 64 631 11]
 [664 29 638 11]] 



[[0 0 183 10]
 [1 3 183 10]
 [2 5 184 10]
 ...
 [494 2335 118 15]
 [495 2331 118 15]
 [496 2327 118 15]] 



[[0 0 183 10]
 [1 2 183 10]
 [2 4 183 10]
 ...
 [493 2